# 安徽安庆市项目4月`CMAQ-ISAM`
## 利用`MCIP`生成的`IOAPI`制作`GridMask.nc`

---
*@author: Evan*\
*@date: 2023-04-30*

In [1]:
import xarray as xr
import numpy as np
import netCDF4 as nc

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
ds = xr.open_dataset('./input/GRIDCRO2D_2023100.nc')
ds

<xarray.Dataset>
Dimensions:  (TSTEP: 1, VAR: 7, DATE-TIME: 2, LAY: 1, ROW: 138, COL: 135)
Dimensions without coordinates: TSTEP, VAR, DATE-TIME, LAY, ROW, COL
Data variables:
    TFLAG    (TSTEP, VAR, DATE-TIME) int32 ...
    LAT      (TSTEP, LAY, ROW, COL) float32 ...
    LON      (TSTEP, LAY, ROW, COL) float32 ...
    MSFX2    (TSTEP, LAY, ROW, COL) float32 ...
    HT       (TSTEP, LAY, ROW, COL) float32 ...
    DLUSE    (TSTEP, LAY, ROW, COL) float32 ...
    LWMASK   (TSTEP, LAY, ROW, COL) float32 ...
    PURB     (TSTEP, LAY, ROW, COL) float32 ...
Attributes: (12/33)
    IOAPI_VERSION:  $Id:: init3.F 178 2015-03-02 16:35:15Z coats             ...
    EXEC_ID:        mcip                                                     ...
    FTYPE:          1
    CDATE:          2023116
    CTIME:          130610
    WDATE:          2023116
    ...             ...
    VGLVLS:         [1.     0.9979]
    GDNAM:          CN3AH_135X138_CR
    UPNAM:          OUTGM3IO        
    VAR-LIST:       LAT             LON             MSFX2           HT       ...
    FILEDESC:       US EPA COMMUNITY MULTISCALE AIR QUALITY MODEL            ...
    HISTORY:

In [3]:
# 将部分变量修改为追踪的地区
# ds = ds.rename_vars({'PRSFC':'Anqing',
#                      'USTAR':'Hefei',
#                      'WSTAR':'Tongling',
#                      'PBL':'Chizhou',
#                      'ZRUF':'Jiujiang',
#                      'MOLI':'Wuhu',
#                      'HFX':'Maanshan'})

ds = ds.rename_vars({'MSFX2':'Anqing',
                     'HT':'Hefei',
                     'DLUSE':'Tongling',
                     'LWMASK':'Chizhou',
                     'PURB':'Jiujiang',})

# 将修改后的变量名写入VAR-LIST并保证字符数为16
var_list = [var_name.ljust(16) for var_name in ds.data_vars.keys()]
ds.attrs['VAR-LIST'] = ''.join(var_list)

ds
# # 将时间维度降为1，否则CCTM会报错
# ds_new = ds.isel(TSTEP=0).expand_dims('TSTEP')
# ds_new

<xarray.Dataset>
Dimensions:   (TSTEP: 1, VAR: 7, DATE-TIME: 2, LAY: 1, ROW: 138, COL: 135)
Dimensions without coordinates: TSTEP, VAR, DATE-TIME, LAY, ROW, COL
Data variables:
    TFLAG     (TSTEP, VAR, DATE-TIME) int32 ...
    LAT       (TSTEP, LAY, ROW, COL) float32 ...
    LON       (TSTEP, LAY, ROW, COL) float32 ...
    Anqing    (TSTEP, LAY, ROW, COL) float32 ...
    Hefei     (TSTEP, LAY, ROW, COL) float32 ...
    Tongling  (TSTEP, LAY, ROW, COL) float32 ...
    Chizhou   (TSTEP, LAY, ROW, COL) float32 ...
    Jiujiang  (TSTEP, LAY, ROW, COL) float32 ...
Attributes: (12/33)
    IOAPI_VERSION:  $Id:: init3.F 178 2015-03-02 16:35:15Z coats             ...
    EXEC_ID:        mcip                                                     ...
    FTYPE:          1
    CDATE:          2023116
    CTIME:          130610
    WDATE:          2023116
    ...             ...
    VGLVLS:         [1.     0.9979]
    GDNAM:          CN3AH_135X138_CR
    UPNAM:          OUTGM3IO        
    VAR-LIST:       TFLAG           LAT             LON             Anqing   ...
    FILEDESC:       US EPA COMMUNITY MULTISCALE AIR QUALITY MODEL            ...
    HISTORY:

In [4]:
import geopandas as gpd
import shapely.geometry as sgeom
from shapely.prepared import prep

def polygon_to_mask(polygon, x, y):
    '''
    Generate a mask array of points falling into the polygon
    '''
    x = np.atleast_1d(x)
    y = np.atleast_1d(y)
    mask = np.zeros(x.shape, dtype=bool)

    # if each point falls into a polygon, without boundaries
    prepared = prep(polygon)
    for index in np.ndindex(x.shape):
        point = sgeom.Point(x[index], y[index])
        if prepared.contains(point):
            mask[index] = True

    return mask

In [6]:
lat = ds.LAT.squeeze()
lon = ds.LON.squeeze()

In [8]:
%%time

cities = ['Anqing', 'Hefei', 'Tongling', 'Chizhou', 'Jiujiang',] # 'Wuhu', 'Maanshan']

for city in cities:
    shp = gpd.read_file(f'./shp/{city}.shp')
    for i in range(np.size(ds.ROW)):
        for j in range(np.size(ds.COL)):
            if polygon_to_mask(shp.geometry[0], lon[i, j], lat[i, j]) == False:
                ds[f'{city}'][:, :, i, j] = 0
            else:
                ds[f'{city}'][:, :, i, j] = 1

CPU times: total: 59.2 s
Wall time: 1min 2s


In [11]:
ds['LAT'] = ds.LAT*0
ds['LON'] = ds.LON*0

In [12]:
# 保留IOAPI格式导出
ds.to_netcdf('D:/Download/GRIDMASK_xarray.nc', format='NETCDF3_CLASSIC')

In [3]:
ds = ds.drop_vars(['MSFX2','HT','DLUSE','LWMASK','PURB'])
ds

<xarray.Dataset>
Dimensions:   (TSTEP: 1, VAR: 7, DATE-TIME: 2, LAY: 1, ROW: 138, COL: 135)
Dimensions without coordinates: TSTEP, VAR, DATE-TIME, LAY, ROW, COL
Data variables:
    TFLAG     (TSTEP, VAR, DATE-TIME) int32 ...
    LAT       (TSTEP, LAY, ROW, COL) float32 ...
    LON       (TSTEP, LAY, ROW, COL) float32 ...
    Anqing    (TSTEP, LAY, ROW, COL) float32 ...
    Hefei     (TSTEP, LAY, ROW, COL) float32 ...
    Tongling  (TSTEP, LAY, ROW, COL) float32 ...
    Chizhou   (TSTEP, LAY, ROW, COL) float32 ...
    Jiujiang  (TSTEP, LAY, ROW, COL) float32 ...
Attributes: (12/33)
    IOAPI_VERSION:  $Id:: init3.F 178 2015-03-02 16:35:15Z coats             ...
    EXEC_ID:        mcip                                                     ...
    FTYPE:          1
    CDATE:          2023116
    CTIME:          130610
    WDATE:          2023116
    ...             ...
    VGLVLS:         [1.     0.9979]
    GDNAM:          CN3AH_135X138_CR
    UPNAM:          OUTGM3IO        
    VAR-LIST:       LAT             LON             Anqing          Hefei    ...
    FILEDESC:       US EPA COMMUNITY MULTISCALE AIR QUALITY MODEL            ...
    HISTORY:

In [5]:
# from PseudoNetCDF

ds.to_netcdf('D:/Download/file.nc')